In [2]:
import numpy as np
import math

## Reference Data

## Run Data

In [27]:
def max_diff_for(ci_a, ci_b):
    zeros = np.zeros_like(ci_a)
        
    diff = np.where(ci_a>0, ci_a, zeros) - np.where(ci_b>0, ci_b, zeros)
    
    min_diff = np.min(diff)
    max_diff = np.max(diff)
    
    min_pos = np.amin(diff)
    max_pos = np.amax(diff)
    
    print("min", min_diff, "max", max_diff)
    
    return diff, min_diff, max_diff, min_pos, max_pos

In [4]:
import glob, re
import itertools, operator

## Load Layer Data

#### Individual Code

In [5]:
f = glob.glob("./volumes/concat_codes/*1-in-3*rate-layers.npz")[0]
print(f)
data_comp = np.load(f)
ci_comp = data_comp["ci"]

./volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6Q2Nmfgo=.1-in-3.json.gz.tar-rate-layers.npz


## Plot

In [6]:
import mayavi
import mayavi.mlab as mlab

from scipy import spatial
from scipy.ndimage.morphology import binary_dilation

In [7]:
from matplotlib import pyplot

# nonlinear color transformation around midpoint
from matplotlib.colors import SymLogNorm
color_mapper_diff = SymLogNorm(linthresh=.00008, linscale=.00008, vmin=-.1, vmax=.1)
color_mapper_swatch = lambda x: x

from matplotlib import cm

#### Reference Graph

In [8]:
data_ref = np.load("./volumes/single-letter-layers.npz")
ci_ref = data_ref["ci"]
qs_ref = data_ref["qs"]

# transpose from LAYER, nx, ny, coordinates
# to coordinates, LAYER, nx, ny
# so that each of x, y, z are a list of 2d arrays for one plane
# compatible with what ci_ref looks like
x, y, z = qs_ref.transpose((3,0,1,2))

#### Plot Type

In [43]:
# layer plot
diff, min_diff, max_diff, min_pos, max_pos = max_diff_for(ci_comp, ci_ref)

min -0.6666666666666667 max 0.006939535632692599


In [10]:
colorspace = np.linspace(-.1, .1, 255)
viridis = cm.get_cmap('Spectral')
INTERPOLATE = True
color_mapper = color_mapper_diff

In [45]:
ANTIALIAS = 8
LINEWIDTH = 2

mlab.options.offscreen = True
mlab.figure(bgcolor=(1,1,1), size=(512,512))

for (dxx, dyy, dzz, dss) in zip(x, y, z, diff):
    mesh = mlab.mesh(dxx, dyy, dzz, scalars=np.copy(dss), colormap="Spectral", opacity=.95, mask=~binary_dilation(~(dss==0), iterations=10))
    mesh.actor.property.lighting = False
    mesh.actor.mapper.interpolate_scalars_before_mapping = INTERPOLATE
    mesh.module_manager.scalar_lut_manager.lut.table = 255*viridis(color_mapper(colorspace))
    mesh.module_manager.scalar_lut_manager.data_range = (min(colorspace), max(colorspace))
    mesh.module_manager.scalar_lut_manager.lut.nan_color = 0, 0, 0, 0

AXES_RANGE = .52
def draw_line(px, py, pz, color=(0,0,0)):
    mlab.plot3d(px, py, pz, line_width=LINEWIDTH*ANTIALIAS, tube_radius=None, color=color)
    
draw_line((0,AXES_RANGE),(0,0),(0,0))
draw_line((0,0),(0,AXES_RANGE),(0,0))
draw_line((0,0),(0,0),(0,AXES_RANGE))

for i in np.arange(.1, .4, .1):
    #bottom
    draw_line((i,i),(0,AXES_RANGE),(0,0), color=(.75,.75,.75))
    draw_line((0,AXES_RANGE),(i,i),(0,0), color=(.75,.75,.75))
    #left
    draw_line((i,i),(0,0),(0,AXES_RANGE), color=(.75,.75,.75))
    draw_line((0,AXES_RANGE),(0,0),(i,i), color=(.75,.75,.75))
    #right
    draw_line((0,0),(i,i),(0,AXES_RANGE), color=(.75,.75,.75))
    draw_line((0,0),(0,AXES_RANGE),(i,i), color=(.75,.75,.75))


draw_line((.5,.5),(0,AXES_RANGE),(0,0), color=(.5,.5,.5))
draw_line((0,AXES_RANGE),(.5,.5),(0,0), color=(.5,.5,.5))

draw_line((.5,.5),(0,0),(0,AXES_RANGE), color=(.5,.5,.5))
draw_line((0,AXES_RANGE),(0,0),(.5,.5), color=(.5,.5,.5))

draw_line((0,0),(.5,.5),(0,AXES_RANGE), color=(.5,.5,.5))
draw_line((0,0),(0,AXES_RANGE),(.5,.5), color=(.5,.5,.5))

# reference
if False:
    ps = np.arange(0,AXES_RANGE/3,.001)
    
    mesh = spatial.KDTree(qs_comp)
    def min_to_surface(xs, ys, zs):
        pts = np.array([xs,ys,zs]).transpose()
        dists = [
            mesh.query(p)[0] for p in pts
        ]
        i = np.argsort(dists)[:2]
        pa = pts[i[0]]
        pb = pts[i[1]]
        
        if np.linalg.norm(pa) > np.linalg.norm(pb):
            return pa.transpose()
        return pb.transpose()
        
    def draw_cross(xs, ys, zs):
        x, y, z = min_to_surface(xs, ys, zs)
        mlab.points3d(x, y, z, scale_factor=.007*(LINEWIDTH**.25), reset_zoom=False, color=(0,0,0), resolution=20)
        
    #depolarizing
    draw_cross(ps, ps, ps)
    #2-pauli
    draw_cross(ps, 0.*ps, ps)
    #bb84
    draw_cross(ps-ps**2, ps**2, ps-ps**2)

scene = mlab.gcf().scene
scene.parallel_projection = True

/home/jkrb2/opt/anaconda3/envs/coffeecode/lib/python3.7/site-packages/mayavi/core/utils.py:86: RuntimeWarning: All-NaN axis encountered
  res = [float(np.nanmin(x)), float(np.nanmax(x))]
Exception occurred in traits notification handler for object: <mayavi.core.lut_manager.LUTManager object at 0x7f12df3d9170>, trait: data_range, old value: [0. 1.], new value: [nan nan]
Traceback (most recent call last):
  File "/home/jkrb2/opt/anaconda3/envs/coffeecode/lib/python3.7/site-packages/traits/trait_notifiers.py", line 394, in __call__
    self.handler(*args)
  File "/home/jkrb2/opt/anaconda3/envs/coffeecode/lib/python3.7/site-packages/mayavi/core/lut_manager.py", line 434, in _data_range_changed
    assert value[0] <= value[1]
AssertionError


In [48]:
mlab.savefig("/home/jkrb2/Desktop/snapshot.png", magnification=ANTIALIAS)